# Lecture 5-1

## Classification<br>

Spam 메일 or Ham 메일을 분류 <br>
facebook feed : 흥미있는 것만 들어옴 <br>
Credit Card : 기존에 썻던 거랑 다르면 이상하게 생각하는 기능

예를 들어 <br>
시험에 합격되는것을 0(불합격)과 1(합격)로 하였을 때 Linear regression 으로 하면 여러가지 문제가 있다.<br>
Hypothesis도 기존에 하던 H(x) = Wx+b 로하면 정확히 0이나 1로 값이 안나옴

Hypothesis 의 H(x) 를 g(z)로 변경<br>
g(z) = 1/(1+e**-z)<br>
g(z)를 <b>sigmoid</b> 라고 부른다. <br>
or <br>
logistic function 이라고 한다.

새로운 Hypothesis <br>
z = Wx <br>
H(x) = g(z)

# Lecture 5-2

변경된 H(x)로 cost 그래프를 보면 매끄럽지않은걸 볼 수 있다.<br>
처음 시작하는 부분에 따라서 최솟점이 달라질 수가 있다.<br>
기존에 썼던 것 처럼 gradient를 사용할 수 없다.

C(H(x),y)  = -log(H(x)) : y=1 이라고 하면 <br>
H(x) 의 값이 1 이면 (답을 맞춤) -> cost(1) = 0<br>
H(x) 의 값이 0 이면 (답을 틀림) -> cost = 무한값 

C(H(x),y) = -log(1-H(x)) : y=0 이라고 하면 <Br>
H(x) = 0(답을 맞춤) -> cost = 0<br>
H(x) = 1(답을 틀림) -> cost = 무한값  

이러한 두개를 합치면 기존에 했던 모양이 되어 gradient를 사용할 수 있다.

C(H(x),y) = -ylog(H(x))-(1-y)log(1-H(x))

y = 1 이면 <br>
C = -log(H(x))<br>
y = 0 이면 <br>
C = -log(1-H(x))

---

In [1]:
import tensorflow as tf


D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

X = tf.placeholder(tf.float32, shape=[None,2])
Y = tf.placeholder(tf.float32, shape=[None,1]) # shape 주의
w = tf.Variable(tf.random_normal([2,1]),name='weight') # x 가 두개씩 있고 y는 한개
b = tf.Variable(tf.random_normal([1]),name='bias') # 항상 나가는 값의 크기

hypothesis= tf.sigmoid(tf.matmul(X,w)+b)
cost = -tf.reduce_mean(Y*tf.log(hypothesis)+ (1-Y)* tf.log(1-hypothesis))
#C(H(x),y) = -ylog(H(x))-(1-y)log(1-H(x))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

In [11]:
predicted = tf.cast(hypothesis > 0.5 , dtype=tf.float32) # 이것이 True , false로 나옴
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost,train], feed_dict={X: x_data, Y:y_data})
        if step % 200 == 0:
            print(step, cost_val)
    h, c, a = sess.run([hypothesis,predicted,accuracy], feed_dict={X:x_data,Y:y_data})
    print("\nHypothesis : ",h,"\nCorrect (Y): ", c ,"\nAccuracy: ",a)
    

0 0.9646962
200 0.41151157
400 0.3944919
600 0.38000843
800 0.36705872
1000 0.35515693
1200 0.34404957
1400 0.3335944
1600 0.32370454
1800 0.31432214
2000 0.305405
2200 0.29691985
2400 0.28883836
2600 0.28113586
2800 0.2737898
3000 0.26677957
3200 0.26008585
3400 0.25369057
3600 0.24757689
3800 0.24172892
4000 0.23613174
4200 0.23077135
4400 0.22563462
4600 0.22070919
4800 0.21598361
5000 0.21144694
5200 0.20708905
5400 0.20290034
5600 0.19887199
5800 0.19499557
6000 0.19126318
6200 0.18766743
6400 0.18420146
6600 0.18085869
6800 0.17763317
7000 0.174519
7200 0.17151074
7400 0.16860344
7600 0.16579235
7800 0.16307275
8000 0.16044062
8200 0.15789191
8400 0.15542284
8600 0.1530298
8800 0.1507095
9000 0.14845873
9200 0.14627452
9400 0.14415406
9600 0.14209455
9800 0.14009361
10000 0.13814873

Hypothesis :  [[0.02611557]
 [0.15206179]
 [0.28191018]
 [0.79207504]
 [0.9461132 ]
 [0.982385  ]] 
Correct (Y):  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0


---

실제 데이터

In [13]:
import numpy as np

In [18]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',',dtype=np.float32)
# 무수한 값과 마지막에 0이면 당뇨병 X / 1이면 당뇨병 O

In [19]:
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

In [20]:
X = tf.placeholder(tf.float32, shape=[None,8])
Y = tf.placeholder(tf.float32, shape=[None,1]) # shape 주의
w = tf.Variable(tf.random_normal([8,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

hypothesis= tf.sigmoid(tf.matmul(X,w)+b)
cost = -tf.reduce_mean(Y*tf.log(hypothesis)+ (1-Y)* tf.log(1-hypothesis))
#C(H(x),y) = -ylog(H(x))-(1-y)log(1-H(x))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5 , dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    feed = {X: x_data , Y:y_data}
    for step in range(10001):
        sess.run(train, feed_dict=feed)
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict=feed))
    h, c, a = sess.run([hypothesis,predicted,accuracy], feed_dict=feed)
    print("\nHypothesis : ",h,"\nCorrect (Y): ", c ,"\nAccuracy: ",a)
    

0 1.664218
200 1.1658957
400 0.9282303
600 0.83249557
800 0.7826365
1000 0.7461157
1200 0.71540284
1400 0.68866193
1600 0.6651211
1800 0.6442327
2000 0.62554586
2200 0.60868686
2400 0.59335065
2600 0.5792906
2800 0.56630874
3000 0.5542457
3200 0.54297364
3400 0.53238875
3600 0.5224066
3800 0.51295745
4000 0.50398374
4200 0.4954372
4400 0.4872768
4600 0.4794678
4800 0.4719803
5000 0.4647882
5200 0.45786887
5400 0.4512025
5600 0.44477117
5800 0.43855935
6000 0.43255293
6200 0.42673925
6400 0.42110693
6600 0.41564566
6800 0.4103459
7000 0.40519917
7200 0.40019742
7400 0.39533356
7600 0.39060083
7800 0.3859931
8000 0.3815045
8200 0.37712988
8400 0.3728642
8600 0.36870286
8800 0.36464152
9000 0.36067608
9200 0.35680288
9400 0.35301816
9600 0.34931874
9800 0.3457013
10000 0.3421629

Hypothesis :  [[0.780158  ]
 [0.24859631]
 [0.719645  ]
 [0.07262695]
 [0.3238148 ]
 [0.652957  ]
 [0.4492848 ]
 [0.68060994]] 
Correct (Y):  [[1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]] 
Accuracy:  1.0
